In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [2]:
df_italy = gpd.read_file('../data/gadm36_ITA_shp/gadm36_ITA_3.shp')
df_trento = df_italy.loc[df_italy['NAME_2']=='Trento']
df_boundary = df_trento[['NAME_3', 'geometry']]
# df_boundary.to_file('../data/trento.geojson', driver='GeoJSON')

In [3]:
f = h5py.File('d:/trentino_10M.h5', 'r')
df_traffic = f['data'][()]
df_idx = f['idx'][()].astype(str)

In [4]:
df_net = pd.DataFrame(df_traffic[:, :, -1]).T
df_net['cellId'] = df_net.index.values + 1

In [5]:
df_grid = gpd.read_file('../data/trentino-grid.geojson')

In [6]:
df_grid.head()

,cellId,geometry
0,38,"POLYGON ((10.91462 45.68201, 10.92746 45.68179..."
1,39,"POLYGON ((10.92746 45.68179, 10.94029 45.68157..."
2,40,"POLYGON ((10.94029 45.68157, 10.95312 45.68136..."
3,154,"POLYGON ((10.90209 45.69122, 10.91493 45.69100..."
4,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079..."


In [7]:
df_net_geo = pd.merge(left=df_net, right=df_grid, on='cellId')

In [8]:
df_net_geo.head()

,0,1,2,3,4,5,6,7,8,9,...,8920,8921,8922,8923,8924,8925,8926,8927,cellId,geometry
0,11.537291,10.264420,9.793975,9.965242,9.047048,8.782508,9.368376,9.035385,7.831252,10.641047,...,21.844774,17.491506,18.975084,17.103937,17.411309,15.896937,17.962266,14.910954,38,"POLYGON ((10.91462 45.68201, 10.92746 45.68179..."
1,12.843910,12.017436,8.237429,9.254380,8.917248,8.673754,8.754773,7.898159,9.178850,11.133600,...,14.283378,11.713794,14.887374,13.173876,15.371801,11.469751,13.582978,13.392556,39,"POLYGON ((10.92746 45.68179, 10.94029 45.68157..."
2,14.445915,13.061264,11.759167,11.627027,11.033120,10.821007,12.483732,10.165051,9.309483,12.446506,...,14.983351,14.340512,15.816820,14.255538,14.545729,14.832623,15.595674,14.137231,40,"POLYGON ((10.94029 45.68157, 10.95312 45.68136..."
3,20.264727,19.326094,11.616555,11.122925,12.534758,10.536824,11.324676,11.382988,9.771106,12.676495,...,16.007255,12.394459,16.577447,16.487158,18.569156,14.270448,16.204483,17.686014,154,"POLYGON ((10.90209 45.69122, 10.91493 45.69100..."
4,19.117608,18.034870,12.254325,13.927390,13.693712,11.970015,12.968228,11.610426,10.890392,13.939911,...,21.027693,18.066915,21.576927,18.856903,23.278456,17.435099,21.576906,21.094357,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079..."


In [9]:
results = gpd.sjoin(df_boundary, gpd.GeoDataFrame(df_net_geo), how='inner', op='intersects')

In [10]:
final_results = results.drop(['index_right'], axis=1)

In [11]:
final_results.head()

,NAME_3,geometry,0,1,2,3,4,5,6,7,...,8919,8920,8921,8922,8923,8924,8925,8926,8927,cellId
7130,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",20.386062,27.667267,22.184880,20.491984,17.705637,18.720098,18.506888,15.886946,...,24.995312,25.180434,35.638643,33.296576,22.406444,41.273252,25.054219,35.272228,23.225812,1453
7130,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",15.584599,17.782143,18.041356,15.400904,12.694927,13.853782,14.033542,11.662857,...,20.502425,20.357764,32.820201,30.885904,19.717368,33.021472,21.334606,29.273747,18.307306,1452
7130,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",10.312440,9.017087,10.555682,9.436423,7.511089,7.339026,8.160558,7.090673,...,14.782503,14.903968,18.042539,17.187629,14.159110,17.220199,14.887728,16.205539,11.713479,1451
7130,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",13.521709,12.797758,10.341977,12.462444,8.306805,7.152446,9.200659,8.354153,...,15.685632,15.754662,15.195472,15.097424,14.391674,14.284066,14.597349,10.951982,8.045141,1450
7130,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",17.086404,16.744265,13.092822,15.964747,10.020006,8.756446,11.412308,10.228674,...,20.765666,20.557188,20.526880,19.632415,18.871101,19.211134,18.996794,13.635104,9.925692,1449


In [12]:
final_grouped = final_results.groupby('NAME_3').agg('sum').reset_index().iloc[:, np.r_[0, 1:8929]]

In [13]:
final_grouped.head()

,NAME_3,0,1,2,3,4,5,6,7,8,...,8918,8919,8920,8921,8922,8923,8924,8925,8926,8927
0,Ala,1834.279605,1799.292406,1659.612163,1759.485205,1664.006275,1589.208664,1725.167979,1494.564057,1523.744861,...,2324.691128,2369.481252,2407.757515,3699.183063,3356.830207,2275.641202,3722.331464,2233.791588,3388.531641,2030.196006
1,Albiano,455.635826,396.884210,243.073130,236.034766,225.087587,200.848176,225.597121,191.403951,175.473176,...,390.267438,348.654859,331.141911,593.787405,544.245978,350.848623,498.009017,289.713957,466.285167,308.643132
2,Aldeno,486.299601,396.688551,377.103091,390.091502,357.258287,344.810159,357.937077,336.598562,323.322654,...,446.092599,486.006142,466.153640,808.486915,700.672426,477.691799,655.072395,435.935807,632.130221,404.678481
3,Amblar,274.785413,319.683966,309.802639,330.988328,303.553711,317.921872,287.285611,347.803704,411.187909,...,705.739473,762.913482,736.626533,1258.444413,993.195974,692.566690,1200.716551,480.575636,1019.427973,656.758595
4,Andalo,477.933221,280.162373,266.923707,250.150824,608.760961,442.209771,229.006385,231.566625,223.797620,...,1771.668195,1821.774386,1714.628673,2160.962485,2105.764831,1687.104011,2172.551486,1877.045612,2125.056604,1822.548558


In [14]:
final_grouped.index = final_grouped.NAME_3.values

In [15]:
final_traffic = final_grouped.iloc[:, 1:8929]

In [16]:
final_traffic.head()

,0,1,2,3,4,5,6,7,8,9,...,8918,8919,8920,8921,8922,8923,8924,8925,8926,8927
Ala,1834.279605,1799.292406,1659.612163,1759.485205,1664.006275,1589.208664,1725.167979,1494.564057,1523.744861,1629.033531,...,2324.691128,2369.481252,2407.757515,3699.183063,3356.830207,2275.641202,3722.331464,2233.791588,3388.531641,2030.196006
Albiano,455.635826,396.884210,243.073130,236.034766,225.087587,200.848176,225.597121,191.403951,175.473176,228.741123,...,390.267438,348.654859,331.141911,593.787405,544.245978,350.848623,498.009017,289.713957,466.285167,308.643132
Aldeno,486.299601,396.688551,377.103091,390.091502,357.258287,344.810159,357.937077,336.598562,323.322654,409.760532,...,446.092599,486.006142,466.153640,808.486915,700.672426,477.691799,655.072395,435.935807,632.130221,404.678481
Amblar,274.785413,319.683966,309.802639,330.988328,303.553711,317.921872,287.285611,347.803704,411.187909,296.949932,...,705.739473,762.913482,736.626533,1258.444413,993.195974,692.566690,1200.716551,480.575636,1019.427973,656.758595
Andalo,477.933221,280.162373,266.923707,250.150824,608.760961,442.209771,229.006385,231.566625,223.797620,230.327142,...,1771.668195,1821.774386,1714.628673,2160.962485,2105.764831,1687.104011,2172.551486,1877.045612,2125.056604,1822.548558


In [17]:
traffic_nid = final_traffic.T

In [19]:
time_index = pd.to_datetime(df_idx, format='%Y-%m-%d %H:%M')

In [20]:
traffic_nid.index = time_index.values

In [21]:
traffic_nid.head()

,Ala,Albiano,Aldeno,Amblar,Andalo,Arco,Avio,Baselga di Pinè,Bedollo,Bersone,...,Vigo Di Fassa,Vigo Rendena,Vigolo Vattaro,Villa Agnedo,Villa Lagarina,Villa Rendena,Volano,Zambana,Ziano Di Fiemme,Zuclo
2013-11-01 00:00:00,1834.279605,455.635826,486.299601,274.785413,477.933221,2161.749261,1239.562563,1095.196490,350.782913,102.385216,...,327.246980,211.952106,966.640029,187.841433,1326.826898,313.287531,686.122382,666.003785,409.268007,292.645475
2013-11-01 00:10:00,1799.292406,396.884210,396.688551,319.683966,280.162373,1919.867024,1305.504866,998.399572,305.356641,103.657035,...,271.902230,197.749213,877.811857,159.654955,1260.653278,302.691479,581.265269,610.997426,457.138274,290.196379
2013-11-01 00:20:00,1659.612163,243.073130,377.103091,309.802639,266.923707,2036.553265,1117.784068,986.154678,328.348339,104.027529,...,275.348665,240.599335,797.659481,187.407055,1353.188203,351.580572,569.341719,596.266525,524.632477,322.683707
2013-11-01 00:30:00,1759.485205,236.034766,390.091502,330.988328,250.150824,1880.307431,1343.197738,930.855022,300.981347,103.428781,...,235.950984,217.635830,796.748938,179.787417,1189.660157,331.720628,571.187055,516.170110,447.970954,339.314818
2013-11-01 00:40:00,1664.006275,225.087587,357.258287,303.553711,608.760961,1908.903639,1189.486184,903.612336,319.421175,106.612852,...,205.551884,158.121196,708.462242,154.730143,1198.917118,263.085283,550.201356,609.074256,431.504766,312.662955


In [22]:
traffic_nid_pd = pd.DataFrame(traffic_nid)
traffic_nid_pd.to_csv('../data/trentino_traffic_nid.csv')

In [23]:
df_corr = traffic_nid_pd.corr().abs()

In [24]:
df_corr_trento = df_corr['Trento']

In [25]:
df_corr_trento

Ala                0.650676
Albiano            0.510934
Aldeno             0.810378
Amblar             0.399017
Andalo             0.036331
                     ...   
Villa Rendena      0.353175
Volano             0.825636
Zambana            0.678972
Ziano Di Fiemme    0.263651
Zuclo              0.683521
Name: Trento, Length: 223, dtype: float64

In [26]:
df_boundary_corr = pd.merge(left=df_boundary, right=df_corr_trento, left_on='NAME_3', right_on=df_corr_trento.index)

In [27]:
df_boundary_corr

,NAME_3,geometry,Trento
0,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",0.650676
1,Albiano,"POLYGON ((11.20962 46.14533, 11.20694 46.14315...",0.510934
2,Aldeno,"POLYGON ((11.07068 45.96799, 11.07569 45.97525...",0.810378
3,Amblar,"POLYGON ((11.18995 46.36033, 11.18942 46.36097...",0.399017
4,Andalo,"MULTIPOLYGON (((11.02194 46.15219, 11.01475 46...",0.036331
...,...,...,...
218,Villa Rendena,"POLYGON ((10.58591 46.06954, 10.58489 46.07254...",0.353175
219,Volano,"POLYGON ((11.11631 45.90346, 11.11498 45.90248...",0.825636
220,Zambana,"MULTIPOLYGON (((11.08496 46.18270, 11.09377 46...",0.678972
221,Ziano Di Fiemme,"POLYGON ((11.55856 46.32783, 11.56022 46.32756...",0.263651


In [28]:
df_boundary_corr.to_file('../data/trento_corr.geojson', driver='GeoJSON')

In [29]:
df_net_avg = pd.DataFrame(traffic_nid_pd.mean(), columns=['avg_traffic'])

In [31]:
df_net_avg.head()

,avg_traffic
Ala,1954.280132
Albiano,309.996382
Aldeno,514.661287
Amblar,403.118965
Andalo,461.355459


In [32]:
df_boundary_traffic = pd.merge(left=df_boundary, right=df_net_avg, left_on='NAME_3', right_on=df_net_avg.index)

In [33]:
df_boundary_traffic.head()

,NAME_3,geometry,avg_traffic
0,Ala,"POLYGON ((11.13952 45.69776, 11.13701 45.69660...",1954.280132
1,Albiano,"POLYGON ((11.20962 46.14533, 11.20694 46.14315...",309.996382
2,Aldeno,"POLYGON ((11.07068 45.96799, 11.07569 45.97525...",514.661287
3,Amblar,"POLYGON ((11.18995 46.36033, 11.18942 46.36097...",403.118965
4,Andalo,"MULTIPOLYGON (((11.02194 46.15219, 11.01475 46...",461.355459


In [34]:
df_boundary_traffic.to_file('../data/trentino_avg_traffic.geojson', driver='GeoJSON')